# Shor's factoring algorithm

In [1]:
from qsystem import QSystem, Gates
from random import randint
from math import log2, ceil, floor, gcd
from IPython.display import display, Latex, Math

In [2]:
N = 15
Latex('Factoring $N = {}$'.format(N))

<IPython.core.display.Latex object>

## step 1: Random select an $a$ less than $N$ and coprime with $N$

In [3]:
a = 0
while gcd(N, a) != 1 or a == 1:
    a = randint(2, N)
Math('a = {}'.format(a))

<IPython.core.display.Math object>

## step 2: use the quantum period-finding to find the period $r$ of the function $f(x) = a^x \mod N$

### Period-finding

It will be necessary 2 quantum registers with size of 
$$n = \lceil\log_2(N+1)\rceil$$

In [4]:
n = ceil(log2(N+1))
Math('n = {}'.format(n))

<IPython.core.display.Math object>

and a quantum oracle 'POWN' that 
$$\left|x\right>\left|0\right>
\xrightarrow{\text{POWN}}
\left|x\right>\left|a^x (\text{mod}\,N)\right>$$

In [5]:
gates = Gates()        

def pown(x):
    x = x >> n
    fx = pow(a, x, N)
    return (x << n) | fx

def it():
    for x in range(2**n):
        yield x << n
    
gates.make_fgate('POWN', pown, 2*n, it())

In [6]:
seed = randint(0,10000)
q = QSystem(n, gates, seed)

print('init first register at')
print(q)

init first register at
+1.000       |0000>



### step 1: Prepare a superposition
$$\left|0\right>\left|0\right>
\xrightarrow{H^{\otimes n}}
{1\over\sqrt{2^n}}\sum_{x=0}^{2^{n}-1} \left|x\right>\left|0\right>
$$

In [7]:
q.evol(gate='H', qbit=0, count=n)
print(q)

+0.250       |0000>
+0.250       |0001>
+0.250       |0010>
+0.250       |0011>
+0.250       |0100>
+0.250       |0101>
+0.250       |0110>
+0.250       |0111>
+0.250       |1000>
+0.250       |1001>
+0.250       |1010>
+0.250       |1011>
+0.250       |1100>
+0.250       |1101>
+0.250       |1110>
+0.250       |1111>



### step 2: Prepare a periodic superposition
$$
{1\over\sqrt{2^n}}\sum_{x=0}^{2^{n}-1} \left|x\right>\left|0\right>
\xrightarrow{\text{POWN}}
{1\over\sqrt{2^n}}\sum_{x=0}^{2^{n}-1}
\left|x\right>\left|a^x(\text{mod}\, N)\right>
$$

In [8]:
print('add the second register')
q.add_ancillas(n)
q.evol(gate='POWN', qbit=0)
print(q)

add the second register
+0.250       |0000>|0001>
+0.250       |0001>|1101>
+0.250       |0010>|0100>
+0.250       |0011>|0111>
+0.250       |0100>|0001>
+0.250       |0101>|1101>
+0.250       |0110>|0100>
+0.250       |0111>|0111>
+0.250       |1000>|0001>
+0.250       |1001>|1101>
+0.250       |1010>|0100>
+0.250       |1011>|0111>
+0.250       |1100>|0001>
+0.250       |1101>|1101>
+0.250       |1110>|0100>
+0.250       |1111>|0111>



### step 3 (optional): measure the second register
help to understand the algorithm

$$
{1\over\sqrt{2^n}}\sum_{x=0}^{2^{n}-1}
\left|x\right>\left|a^x(\text{mod}\, N)\right>
\xrightarrow{\text{measure}[n:2n]}
\sqrt{r\over{2^n}}\sum_{i=0}^{{2^{n}\over r}-1}
\left|ir + x_0\right>\left|a^{x_0}(\text{mod}\, N)\right>
$$


In [9]:
print('measure and remove the second register')
q.rm_ancillas() 
print(q)

measure and remove the second register
+0.500       |0000>
+0.500       |0100>
+0.500       |1000>
+0.500       |1100>



### step 4: fourier transform of the first register
$$
\sqrt{r\over{2^n}}\sum_{i=0}^{{2^{n}\over r}-1}
\left|ir + x_0\right>
\xrightarrow{\text{QFT}_n}
{1\over\sqrt{r}}\sum_{i=0}^{r-1}\left|i{2^n\over r}\right>\phi_i
$$

In [10]:
q.qft(qbegin=0, qend=n)
print(q)

+0.500       |0000>
+0.500       |0100>
+0.500       |1000>
+0.500       |1100>



### step 5: measure the first register and repeat the algorithm to measure distincts multiples of $2^n\over r$

In [11]:
q.measure_all()
c = q.bits()
c = sum([m*2**i for m, i in zip(c, reversed(range(len(c))))])
mea = [c]

for _ in range(n-1):
    seed = randint(0,10000)
    q = QSystem(n, gates, seed)

    q.evol('H', 0, n) # 1
    q.add_ancillas(n)
    q.evol('POWN', 0) # 2
    q.rm_ancillas()
    q.qft(0, n)       # 4
    q.measure_all()   # 5

    c = q.bits()
    c = sum([m*2**i for m, i in zip(c, reversed(range(len(c))))])
    mea.append(c)
print('measurements results =', mea)

measurements results = [12, 8, 8, 4]


### step 6: with the measurements compute 
$$
r = {2^n\over\gcd(\text{measurements})}
$$

In [12]:
c = mea[0]
for m in mea:
    c = gcd(c, m)
if c == 0:
    print('repite the period-finding algorithm')
else:
    r = 2**n/c
    display(Latex('possible period $r = {}$'.format(r)))

<IPython.core.display.Latex object>

## step 3: if $r$ is odd go to step 1 else compute the two nontrivial factors of $N$, $pq = N$
$$
p = \gcd(a^{r\over2}-1, N)
$$
$$
q = \gcd(a^{r\over2}+1, N)
$$

In [13]:
if c == 0:
    print('repite the period-finding algorithm')
elif r % 2 == 1:
    print('go to step 1')
else: 
    p = gcd(int(a**(r/2)+1), N)
    q = gcd(int(a**(r/2)-1), N)
    display(Math('{}*{}={}'.format(p,q,p*q)))

<IPython.core.display.Math object>